<a href="https://colab.research.google.com/github/sbertolin/03MAIR-Algoritmos-de-optimizacion/blob/master/AG2/Santiago_Bertol%C3%ADn_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AG2** - Actividad Guiada 2<br>
Santiago Bertolín Martínez <br>
Url -> https://github.com/sbertolin/03MAIR-Algoritmos-de-optimizacion/tree/master/AG2

In [0]:
import math
import random

In [0]:
from time import time

# Función para calcular el tiempo de ejecución
def calcular_tiempo(f):
   
    def wrapper(*args, **kwargs):        
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper

In [0]:
# Definición función de cálculo de la distancia
def distance(A,B):
  if type(A) is int or type(A) is float:
    return abs(B-A)
  else:
    return math.sqrt(sum([ (A[i]-B[i])**2 for i in range(len(A)) ]))

In [0]:
# Resolución del problema por fuerza Bruta
def brute_force_distance(point_list):
  best_found = 100000e10
  A,B = (),()
  length = len(point_list)
  
  for i in range(length):
    for j in range(i+1, length):
      calc_distance = distance(point_list[i], point_list[j])
      if calc_distance < best_found:
        A, B = point_list[i], point_list[j]
        best_found = calc_distance
        
  return (best_found, [A,B])

In [0]:
@calcular_tiempo
def test_brute_force_distance(list_Xd):
  return brute_force_distance(list_Xd)

In [0]:
# Resolución del problema utilizando Divide y vencerás tal como se ha realizado
# en la actividad guiada
def divide_and_conquer(point_list):
  if len(point_list) < 10:
    return brute_force_distance(point_list)
  
  # sorted usa quick_sort
  sorted_list = sorted(point_list, key=lambda x: x[0])
  low_list = sorted_list[:len(point_list)//2]
  high_list = sorted_list[len(point_list)//2:]
  
  d_low, low_points = divide_and_conquer(low_list)
  d_high, high_points = divide_and_conquer(high_list)
  
  return brute_force_distance(low_points + high_points)

In [0]:
@calcular_tiempo
def test_divide_and_conquer(list_Xd):
  return divide_and_conquer(list_Xd)

In [0]:
# Resolución del problema utilizando una versión optimizada del Divide y vencerás
# realizado en la actividad guiada
def divide_and_conquer_mod(point_list, is_init = True):
  list_length = len(point_list)
  if list_length < 6:
    return brute_force_distance(point_list)
  
  # Una vez ordenadas las listas, no hace falta volverlas a ordenar
  if is_init:
    point_list = sorted(point_list, key=lambda x: x[0])
    
  low_list = point_list[:list_length//2]
  high_list = point_list[list_length//2:]
  
  d_low, low_points = divide_and_conquer_mod(low_list, False)
  d_high, high_points = divide_and_conquer_mod(high_list, False)
  
  return brute_force_distance(low_points + high_points)

In [0]:
@calcular_tiempo
def test_divide_and_conquer_mod(list_Xd):
  return divide_and_conquer_mod(list_Xd, True)

In [0]:
# Resolución del problema utilizando una versión optimizada del Divide y vencerás
# y resolviendo el posible error en caso de encontrar la solución partida entre las
# dos listas de una de las iteraciónes del divide y vencerás
def divide_and_conquer_mod_corrected(point_list, is_init = True):
  list_length = len(point_list)
  if list_length < 20:
    return brute_force_distance(point_list)
  
  # Una vez ordenadas las listas, no hace falta volverlas a ordenar
  if is_init:
    point_list = sorted(point_list, key=lambda x: x[0])
    
  low_list = point_list[:list_length//2]
  high_list = point_list[list_length//2:]
  
  d_low, low_points = divide_and_conquer_mod_corrected(low_list, False)
  d_high, high_points = divide_and_conquer_mod_corrected(high_list, False)

  if d_low > d_high:
    d_low = d_high

  # Se buscan los puntos próximos a la intersección entre las listas que cumplan
  # que su distancia en el eje de comparación sea el menor que la distancia mínima
  # encontrada hasta el momento.
  length_low = len(low_list)
  low_iterator = length_low-1
  high_iterator = 0
  check_list = []
  
  # Se recorre la lista superior y se almacenan todos los puntos que cumplen que
  # su distancia en el primer eje és menor con el último punto de la lista inferior
  tmp_val = low_list[length_low-1][0] + d_low
  while high_list[high_iterator][0] < tmp_val:
    check_list.append(high_list[high_iterator])
    high_iterator += 1
    if high_iterator == len(high_list):
      break
      
  # Se recorre la lista inferior y se almacenan todos los puntos que cumplen que
  # su distancia en el primer eje és menor con el primer punto de la lista superior
  tmp_val = high_list[0][0] - d_low
  while low_list[low_iterator][0] > tmp_val:
    check_list.insert(0, low_list[low_iterator])
    low_iterator -= 1
    if low_iterator < 0:
      break
  
  # En caso de haber encontrado puntos en la franja problemática, se calcula su
  # distancia mínima y en caso de ser menor a la encontrada en las dos listas,
  # se devuelve en su lugar
  check_lenght = len(check_list)
  if check_lenght > 0:
    if (check_lenght < 20 or check_lenght == list_length):
      d_min2, min_points = brute_force_distance(check_list)
    else:
      d_min2, min_points = divide_and_conquer_mod_corrected(check_list, False)
    if d_min2 <= d_low:
      return d_min2, min_points
  
  return brute_force_distance(low_points + high_points)

In [0]:
@calcular_tiempo
def test_divide_and_conquer_mod_corrected(list_Xd):
  return divide_and_conquer_mod_corrected(list_Xd, True)

In [163]:
# Se generan las constantes, el tamaño de la lista y los puntos de la lista (de forma aleatoria)
N = 3000
list_2d = [(random.randrange(1, N), random.randrange(1, N)) for _ in range(N)]
list_3d = [(random.randrange(1, N), random.randrange(1, N), random.randrange(1, N)) for _ in range(N)]

print("TEST EN 2D")
print("\nFuerza Bruta:")
print(test_brute_force_distance(list_2d))
print("\nDivide y vencerás guiado:")
print(test_divide_and_conquer(list_2d))
print("\nDivide y vencerás optimizado:")
print(test_divide_and_conquer_mod(list_2d))
print("\nDivide y vencerás corregido:")
print(test_divide_and_conquer_mod_corrected(list_2d))

print("\n\nTEST EN 3D")
print("\nFuerza Bruta:")
print(test_brute_force_distance(list_3d))
print("\nDivide y vencerás guiado:")
print(test_divide_and_conquer(list_3d))
print("\nDivide y vencerás optimizado:")
print(test_divide_and_conquer_mod(list_3d))
print("\nDivide y vencerás corregido:")
print(test_divide_and_conquer_mod_corrected(list_3d))

TEST EN 2D

Fuerza Bruta:
Tiempo de ejecución para algoritmo: 9.563021659851074
(1.0, [(694, 1280), (694, 1281)])

Divide y vencerás guiado:
Tiempo de ejecución para algoritmo: 0.030988693237304688
(1.0, [(694, 1280), (694, 1281)])

Divide y vencerás optimizado:
Tiempo de ejecución para algoritmo: 0.029427528381347656
(1.0, [(694, 1280), (694, 1281)])

Divide y vencerás corregido:
Tiempo de ejecución para algoritmo: 0.10739469528198242
(1.0, [(694, 1280), (694, 1281)])


TEST EN 3D

Fuerza Bruta:
Tiempo de ejecución para algoritmo: 11.066687822341919
(18.681541692269406, [(2812, 353, 391), (2816, 335, 394)])

Divide y vencerás guiado:
Tiempo de ejecución para algoritmo: 0.03439974784851074
(19.261360284258224, [(1817, 2111, 201), (1818, 2102, 184)])

Divide y vencerás optimizado:
Tiempo de ejecución para algoritmo: 0.02915167808532715
(19.519221295943137, [(603, 174, 1264), (605, 163, 1280)])

Divide y vencerás corregido:
Tiempo de ejecución para algoritmo: 0.8912544250488281
(18.68154

In [146]:
list_test = list_2d
print(test_divide_and_conquer_mod_corrected(list_test))

Tiempo de ejecución para algoritmo: 0.16482925415039062
(5.656854249492381, [(16947, 24067), (16951, 24063)])


In [0]:
# Problema: Paseo por el rio

# Función para describir la ruta óptima entre dos puntos
def describe_route(route, _from, to):
  if _from == to:
    return _from
  else:
    return str(describe_route(route, _from, route[_from][to])) + ',' + str(route[_from][to])

# Matriz de coste entre los distintos puntos
TARIFFS = [
    [0,  5,  4,  3,  999,999,999],
    [999,0,  999,2,  3,  999,11],
    [999,999,0,  1,  999,4,  10],
    [999,999,999,0,  5,  6,  9],
    [999,999,999,999,0,  999,4],
    [999,999,999,999,999,0,  3],
    [999,999,999,999,999,999,0]]

def prices(tariffs):
  length = len(tariffs[0])
  
  price_arr = [ [9999]*length for i in [9999]*length ]
  route = [ [""]*length for i in [9999]*length ]
  
  for i in range(length-1):
    for j in range(i+1, length):
      MIN = tariffs[i][j]
      route[i][j] = i
      
      for k in range(i,j):
        if price_arr[i][k] + tariffs[k][j] < MIN:
          MIN = min(MIN, price_arr[i][k] + tariffs[k][j])
          route[i][j] = k
          
      price_arr[i][j] = MIN
      
  return price_arr, route

In [194]:
ret_prices, ret_routes = prices(TARIFFS)
print(ret_prices)
print(ret_routes)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


In [0]:
def print_route(origin, destination):
  print("La ruta óptima entre " + str(origin) + " y " + str(destination) + " es:")
  return describe_route(ret_routes, origin, destination)

In [196]:
print_route(0,6)

La ruta óptima entre 0 y 6 es:


'0,0,2,5'